# Zero Shot Classification on Comics dataset

In [1]:
import os
import json
import torch
import pickle

import pandas as pd

from tqdm.notebook import tqdm
from pathlib import Path
from sklearn.metrics import classification_report
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [102]:
# def generate_prompt(text, labels):
#     prompt = f"Classify the following text into one of these categories. Restrict your answer to only one token. Do not generate any tokens other than the label: {', '.join(labels)}.\n\nText: {text}\n\nCategory:"
#     return prompt


In [26]:
def generate_prompt(text, labels):
    prompt = f"You are an expert in emotion analysis. You are given a text from a comics book. ### Here is the text: {text}\n\nYou must classify the emotion of this text into one of the following emotions classes: {labels}. You must return only a single token to fill in the emotion label of the text. Answer:"
    return prompt


In [348]:
# def generate_prompt(text, labels):
#     prompt = f"""You are an expert in emotion analysis. You are given a text from a comics book.\n\n### Here is the text: {text}\n\nYou must classify the emotion of this text into one or more of the following emotion labels: {labels}. Do not generate any explanation after the answer. ###Answer:###"""
#     return prompt

In [5]:
# def generate_prompt(text, labels):
#     prompt = (
#         f"You are an expert in emotion analysis. Below is a passage from a comic book.\n\n"
#         f"### Text:\n{text}\n\n"
#         f"Classify the emotion(s) of the text using one of the following labels: anger, surprise, fear, disgust, sadness, joy, neutral. Provide only the labels.\n\n"
#         f"### Answer:"
#     )
#     return prompt


In [27]:
text_to_classify = "The stock market saw a sharp rise today."
labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

In [28]:
generate_prompt(text_to_classify, labels)

"You are an expert in emotion analysis. You are given a text from a comics book. ### Here is the text: The stock market saw a sharp rise today.\n\nYou must classify the emotion of this text into one of the following emotions classes: ['anger', 'surprise', 'fear', 'disgust', 'sadness', 'joy', 'neutral']. You must return only a single token to fill in the emotion label of the text. Answer:"

In [30]:
def classify_text(text, labels):
    # Create the prompt
    prompt = generate_prompt(text, labels)
    
    # Tokenize and encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate output
    outputs = model.generate(**inputs, max_new_tokens=1, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id, no_repeat_ngram_size=2)
    
    # Decode the generated output
    classification = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the predicted category (after the prompt)
    #predicted_category = classification.split("Answer:")[-1].strip()
    return classification
    #return predicted_category

In [61]:
# text_to_classify = "The stock market saw a sharp rise today."
# labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

In [62]:
#predicted_category = classify_text(text_to_classify, labels)

In [63]:
#print(f"Predicted category: {predicted_category}")

Predicted category: finance


### Read the data sets

In [10]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/zeroshot/datasets/comics_data_processed.csv")

In [12]:
df = df.drop(columns=[df.columns[0], df.columns[1]])

In [14]:
emotion_map = {
    'AN': 'anger',
    'DI': 'disgust',
    'FE': 'fear',
    'SA': 'sadness',
    'SU': 'surprise',
    'JO': 'joy'
}

In [15]:
def extract_emotions(row):

    emotion_str = row.emotion

    if emotion_str == 'Neutral':
        return ['neutral']

    emotions = emotion_str.split('-')
    tags = []

    for emotion in emotions:
        abbrev = emotion[:2]  # Get the abbreviation
        value_part = emotion[2:]  # Get the value part
        
        # Ensure that the value part is a valid integer and abbrev is in the emotion_map
        if abbrev in emotion_map and value_part.isdigit():
            value = int(value_part)
            if value > 0:
                tags.append(emotion_map[abbrev].lower())
        else:
            print(f"Warning: Skipping invalid emotion entry: '{emotion}'")
    return tags  

In [16]:
df['emotions_list'] = df.apply(lambda row: extract_emotions(row), axis=1)

In [18]:
df = df[df['emotions_list'].apply(lambda x: len(x) == 1)].reset_index()

In [31]:
texts = df.utterance.tolist()

In [33]:
labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

In [34]:
predictions = []

for i in tqdm(range(len(texts))):
    predicted_category = classify_text(texts[i], labels)
    predictions.append(predicted_category)

  0%|          | 0/3111 [00:00<?, ?it/s]

In [37]:
preds = []

for prediction in predictions:
    preds.append(prediction.split(" ")[-1])

In [41]:
grounds = [x for [x] in df.emotions_list.tolist()]

In [46]:
for idx, (ground, pred) in enumerate(zip(grounds, preds)):

    if pred not in labels:
        del grounds[idx]
        del preds[idx]

In [48]:
print(classification_report(grounds, preds))

              precision    recall  f1-score   support

       anger       0.64      0.30      0.40       787
     disgust       0.10      0.04      0.06        23
        fear       0.32      0.02      0.05       330
         joy       0.50      0.40      0.45       708
     neutral       0.21      0.41      0.28       341
     sadness       0.51      0.29      0.37       465
    surprise       0.22      0.60      0.32       431

    accuracy                           0.34      3085
   macro avg       0.36      0.30      0.28      3085
weighted avg       0.44      0.34      0.34      3085

